In [4]:
import matplotlib
%matplotlib tk
#matplotlib.use('Agg')

%load_ext autoreload
%autoreload 2

%autosave 180

import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import os

import sys
sys.path.append('..')

# visualize results module
from visualize import visualize_svm as Visualize

from visualize import correlograms

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 180 seconds


<ipython-input-4-9c0a4495f89d>:11: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [8]:
############################################## 
##############################################
##############################################
fname = '/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb16_30Hz/IA1pm_Feb16_30Hz_0secNoMove_movements.npz'
data = np.load(fname, allow_pickle=True)
all_quiescent = data['all_quiescent']
initiations = data['feature_initiations']
quiescent = data['feature_quiescent']
labels = data['labels']
shift = data['video_shift']
print ("labels: ", labels)
print ('initiations: ', initiations.shape)
print ("video shift: ", shift)

# load rewarded lever pulls
rewarded = np.loadtxt('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb16_30Hz/rewarded_times.txt')
print ("# of trials: ", rewarded.shape)
clrs = ['black','magenta','orange']




labels:  ['left_paw' 'right_paw' 'nose' 'jaw' 'right_ear' 'tongue' 'lever']
initiations:  (7, 19954)
video shift:  [4.641 4.915 4.701]
# of trials:  (43,)


In [7]:
############################################## 
##############################################
##############################################
feats = [0,1,5]
t = np.arange(initiations.shape[1])/15.
ctr=0
for k in feats:
    plt.plot(t, initiations[k],c=clrs[ctr],
             linewidth=3,
             label=labels[k],
            alpha=.5)
    ctr+=1
#
plt.scatter(rewarded+shift[0], 
            np.zeros(rewarded.shape[0])+1.25,c='red',
            s=100,
            label = 'lever pulls')
    
plt.legend(fontsize=20)
yticks = ['no movement', 'movement' ]
plt.yticks([0,1],yticks, fontsize=20)
plt.xlabel("Time (sec)", fontsize=20)
plt.xticks(fontsize=20)
#
if False:
    plt.savefig('/home/cat/xcorr.svg')
    plt.close()
else:
    plt.show()
 

In [93]:
############################################## 
##############################################
##############################################
feats = [0,1,5]   # "left paw", "right paw", "tongue"

selected_features = [0,1,2]
random = False
for selected_feature in selected_features:
    # 
    t = np.arange(initiations.shape[1])/15.
    units = [0,1]

    idx = np.where(initiations[feats[selected_feature]]==1)[0]
    print(idx.shape[0])
    spk1 = t[idx]
    spk2 = rewarded + shift[0]

    if random:
        temp2 = np.random.rand(spk1.shape[0])*50
        print (temp2.shape)
        spk1+=temp2    

    #
    spt_all = np.hstack((spk1, spk2))
    unit_ids = np.hstack((np.repeat(0, len(spk1)),
                          np.repeat(1, len(spk2))))

    # 
    order_indexes = np.argsort(spt_all)
    spike_times = spt_all[order_indexes]
    spike_times = np.int64(spike_times * 1E3)
    spike_clusters = unit_ids[order_indexes]

    spike_train = np.vstack((spike_times, spike_clusters)).T
    print ("spike train: ", spike_train.shape)


    #
    sample_rate = 1000
    bin_width = 0.25
    window_size = 15
    soft_assignment  = 	np.ones(len(spike_times))

    #
    cors = correlograms.compute_correlogram(
                                     units,
                                     spike_train,
                                     soft_assignment,
                                     sample_rate=1000,
                                     bin_width = bin_width,
                                     window_size = window_size*2)

    #   
    plt.figure()
    t2 = np.arange(cors.shape[2])*bin_width-window_size
    temp = cors[1,0,:]
    if True:
        plt.plot(t2, temp, 
                c=clrs[selected_feature],
                linewidth=3,
                label = "CrossCorrelation: \nlever vs. " + labels[feats[selected_feature]])
    else:
        idx = np.where(temp<10)
        temp[idx]=np.nan
        plt.scatter(t2, temp, 
             c=clrs[selected_feature],
     #        linewidth=3,
             label = "CrossCorrelation: \nlever vs. " + labels[feats[selected_feature]])
    
        
    
    plt.legend()
    plt.ylim(bottom=0)
    plt.xlabel("Time (sec)")

    plt.savefig('/home/cat/'+str(selected_feature)+'.svg')
    plt.close()
#plt.show()

2635
spike train:  (2645, 2)
bin size:  0.25
3766
spike train:  (3776, 2)
bin size:  0.25
628
spike train:  (638, 2)
bin size:  0.25


In [111]:
feats = [0,1,5]   # "left paw", "right paw", "tongue"

selected_features = [0,1,2]
random = False
animal_id = 'IJ2'

# 
v = Visualize.Visualize()
v.main_dir = '/media/cat/4TBSSD/yuki/'
v.animal_id = animal_id
    
# 
v.session_id = "Mar4"  # use a random date as dummy to load all other sessions
v.get_fname()

for selected_feature in selected_features:
    ctr=0
    min_events=10
    cmap = matplotlib.cm.get_cmap('viridis', 30)
    xcorrs=[]
    #selected_feature = 1
    for session in v.sessions:

        # 
        fname = os.path.join(v.main_dir,
                             v.animal_id,
                             'tif_files',
                             session,
                             session+'_0secNoMove_movements.npz'
                            )

        # 
        try:
            data = np.load(fname, allow_pickle=True)
        except:
            continue

        # 
        all_quiescent = data['all_quiescent']
        initiations = data['feature_initiations']
        quiescent = data['feature_quiescent']
        labels = data['labels']
        shift = data['video_shift']

        # load rewarded lever pulls
        rewarded = np.loadtxt(os.path.join(v.main_dir,
                                          v.animal_id,
                                         'tif_files',
                                         session,
                                        'rewarded_times.txt'))

        # 
        t = np.arange(initiations.shape[1])/15.
        units = [0,1]

        # 
        idx = np.where(initiations[feats[selected_feature]]==1)[0]
        spk1 = t[idx]

        spk2 = rewarded + shift[0]

        #
        if random:
            temp2 = np.random.rand(spk1.shape[0])*50
            spk1+=temp2

        # 
        try:
            if spk1.shape[0]==min_events or spk2.shape[0]==min_events:
                pass

        except:
            continue

        # 
        spt_all = np.hstack((spk1, spk2))
        unit_ids = np.hstack((np.repeat(0, len(spk1)),
                              np.repeat(1, len(spk2))))

        # 
        order_indexes = np.argsort(spt_all)
        spike_times = spt_all[order_indexes]
        spike_times = np.int64(spike_times *1E3)
        spike_clusters = unit_ids[order_indexes]
        spike_train = np.vstack((spike_times, spike_clusters)).T

        #
        sample_rate = 1000
        bin_width = 0.5
        window_size = 15
        soft_assignment = np.ones(len(spike_times))

        #
        cors = correlograms.compute_correlogram(
                                         units,
                                         spike_train,
                                         soft_assignment,
                                         sample_rate=1000,
                                         bin_width = bin_width,
                                         window_size = window_size*2)

        #   
        t2 = np.arange(cors.shape[2])*bin_width-window_size
        try:
            temp = cors[1,0,:]
        except:
            continue

        normalized = True
        if normalized:
            temp = temp/np.max(temp)

        xcorrs.append(temp)

    try:
        xcorrs=np.vstack(xcorrs)
    except:
        continue
    mean = np.mean(xcorrs,axis=0)
    std = np.std(xcorrs,axis=0)
    plt.plot(t2, mean, 
             c=clrs[selected_feature],
             linewidth=3,
             label = "CrossCorrelation: \nlever vs. ",
                    )

    plt.fill_between(t2, mean+std, mean-std,
                     color=clrs[selected_feature],
                     alpha=0.1)

    #plt.legend()
    plt.ylim(bottom=0)
    plt.xlabel("Time (sec)")

    plt.savefig('/home/cat/'+animal_id+" crosscorrelation_longitudinal_"+str(selected_feature)+'.svg')
    plt.close()
#plt.show()

 COULDN't FIND SESSION...
bin size:  0.5
bin size:  0.5
bin size:  0.5
bin size:  0.5
bin size:  0.5
bin size:  0.5
bin size:  0.5
bin size:  0.5
bin size:  0.5
bin size:  0.5
bin size:  0.5
bin size:  0.5
bin size:  0.5
bin size:  0.5
bin size:  0.5
bin size:  0.5
bin size:  0.5
bin size:  0.5
bin size:  0.5
bin size:  0.5
bin size:  0.5
bin size:  0.5
bin size:  0.5
bin size:  0.5
bin size:  0.5
bin size:  0.5
bin size:  0.5
bin size:  0.5
bin size:  0.5
bin size:  0.5
bin size:  0.5
bin size:  0.5
bin size:  0.5
